# Trabajo Práctico Final - MSSCAE 2024 - Grupo 5

---
## ***"Dinámica de formación de precios como fenómeno emergente de la interacción entre agentes económicos"***

#### Integrantes:
* Cubino, Santiago
* Demare, Matías
* Puerta, Ezequiel

---
## Suposiciones del modelo
* Mercado de un único producto.
* Compuesto por agentes *Productores* y *Consumidores*.
* Dispuestos al azar en una grilla cuadrada (Autómata Celular).
* Topología de toroide.
* Los agentes interactúan entre sí para comprar y vender el producto.
* Los agentes interactúan según un vecindario de Moore "extendido".
* El precio del producto se establece como resultado de dichas interacciones.
* El sistema evoluciona a pasos discretos.
* En principio, todos los agentes pueden interactuar en cada iteración.

## Suposiciones de los Productores
Comienzan con:
* Un determinado capital.
* Un stock (no se utilizó por el momento, es virtualmente infinito).
* Un costo fijo de operaciones.
* Un costo marginal por unidad de producto.
* Un período esperado para analizar la ganancia obtenida.
* Un precio inicial (por encima del costo marginal según un *ratio* parametrizado).
* Los Productores pueden endeudarse y permitir que su capital sea negativo (bancarrota).
* Si se desea, el modelo permite eliminar a los Productores en bancarrota.
* En cada iteración el Productor chequea si se ha cumplido el período de ganancia y de ser así, calcula la ganancia obtenida (positiva o negativa) según las ventas del período.
* El rendimiento obtenido en el período impacta directamente en el capital.

## Suposiciones de la formula de ganancia
* La mayoría de los supuestos del Productor sirven para construir la fórmula de ganancia.
* Inicialmente, de manera aleatoria el Productor comienza con una dirección para modificar el precio. Si la ganancia incrementa respecto a la iteración anterior, mantiene la dirección. Caso contrario, la invierte (de aumentar pasa a disminuir el precio, o viceversa).
* Las variaciones del precio son un porcentaje fijo del precio para todos los Productores.

## Suposiciones de los Consumidores
* Deben consumir si o sí en cada iteración.
* No están limitados por capital.
* Compran al Productor que oferte al menor precio del vecindario.

In [36]:
# Vecindario de Moore Extendido n=3
import plotly.graph_objects as go
import numpy as np

size = 11
matrix = np.zeros((size, size))
center = size // 2

for i in range(size):
    for j in range(size):
        distance = max(abs(i - center), abs(j - center))
        if distance == 0:
            matrix[i, j] = 1    # Center (red)
        elif distance <= 3:
            matrix[i, j] = 0.4  # Next 3 rings (light blue)
        else:
            matrix[i, j] = 0.2  # Rest (light green)

producer_positions = [(3, 3), (7, 7), (1, 9), (9, 1)]
for i, j in producer_positions[:2]:
    matrix[i, j] = 0.6  # Soft purple producers inside rings
for i, j in producer_positions[2:]:
    matrix[i, j] = 0  # Dark gray producers outside rings

customdata_matrix = np.full((size, size), "")
text_matrix = np.full((size, size), "Vecindario")
text_matrix[center, center] = "Consumidor"
for i, j in producer_positions[:2]:
    text_matrix[i, j] = "Productores"
    customdata_matrix[i, j] = (i*2)+j
for i, j in producer_positions[2:]:
    text_matrix[i, j] = "Productores inalcanzables"
for i in range(size):
    for j in range(size):
        if max(abs(i - center), abs(j - center)) > 3:
            text_matrix[i, j] = ""

fig = go.Figure(data=go.Heatmap(
    z=matrix,
    text=text_matrix,
    customdata=customdata_matrix,
    hovertemplate='%{text}<br>Precio: %{customdata}<extra></extra>',
    colorscale=[
        [0, 'rgb(64, 64, 64)'],     # Dark gray
        [0.2, 'rgb(200, 255, 200)'],  # Light green
        [0.4, 'rgb(173, 216, 230)'],  # Light blue
        [0.6, 'rgb(180, 120, 220)'],  # Soft purple
        [1, 'rgb(255, 99, 71)']     # Tomato red
    ],
    showscale=False,
))

fig.update_layout(
    width=700,
    height=550,
    title="Vecindario de Moore Extendido n=3",
    xaxis=dict(
        scaleanchor="y",
        scaleratio=1,
        showgrid=False,
        showticklabels=False,
        zeroline=False,
        range=[-0.5, size-0.5]
    ),
    yaxis=dict(
        showgrid=False,
        showticklabels=False,
        zeroline=False,
        range=[-0.5, size-0.5]
    ),
    plot_bgcolor='rgba(0,0,0,0)',
)

for i in range(size + 1):
    fig.add_shape(
        type="line",
        x0=-0.5, y0=i-0.5, x1=size-0.5, y1=i-0.5,
        line=dict(color="black", width=1),
    )
    fig.add_shape(
        type="line",
        x0=i-0.5, y0=-0.5, x1=i-0.5, y1=size-0.5,
        line=dict(color="black", width=1),
    )

legend_colors = ['rgb(255, 99, 71)', 'rgb(173, 216, 230)', 'rgb(180, 120, 220)', 'rgb(64, 64, 64)']
legend_labels = ['Consumidor', 'Vecindario', 'Productores', 'Productores inalcanzables']

for i, (color, label) in enumerate(zip(legend_colors, legend_labels)):
    fig.add_trace(go.Scatter(
        x=[None],
        y=[None],
        mode='markers',
        marker=dict(size=10, color=color),
        showlegend=True,
        name=label
    ))

fig.update_layout(
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=1.05
    )
)
fig.show()

---
## Análisis

### Inicialización

In [1]:
import sys
if '../' not in sys.path:
    sys.path.append('../')

In [9]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from simulab.simulation.core.runner import Runner
from simulab.simulation.core.neighborhood import ExpandedMoore
from simulab.simulation.core.experiment import ExperimentParametersSet
from simulab.simulation.core.equilibrium_criterion import WithoutCriterion

from src.market import Market

from simulab.simulation.plotters.final_grid import FinalGridSeries
from simulab.simulation.plotters.numerical_series import NumericalSeries
from simulab.simulation.plotters.categorical_animated_lattice import (
    CategoricalAnimatedLatticeSeries,
)

### ABC

Vamos a simular el modelo barriendo dos parámetros, el capital y la cantidad de productores en la grilla (mediante la probabilidad de aparición).

In [3]:
experiment_parameters_set = ExperimentParametersSet(
    length=[10],
    neighborhood=[ExpandedMoore(3)],
    agent_types=[2],
    capital=[2500, 100000],
    producer_probability=[0.05, 0.10, 0.50],
    profit_period=[7],
    price_ratio=[(1.2, 1.5)],
    fixed_cost=[(10, 1)],
    marginal_cost=[(10, 1)],
    quantity_to_buy=[(1, 0)],
    bankrupt_enabled=[False],
)
criterion = WithoutCriterion()
runner = Runner(Market, experiment_parameters_set, criterion, max_steps=500)

In [4]:
runner.start()

In [5]:
FinalGridSeries.show_up(
    "price_lattice", 
    runner=runner,
    plot_title=("Evolución de Precios de Mercado"),
    leyend="Precio",
    colorscale="Viridis",
)

Quizas acá se pueden hacer distintos barridos para mostrar los distintos parámetros, y eventualmente quizas llegar a laconclusion que el parámetro mas fuerte es el de porcentaje de productores.

Tambien se podrian graficar algunos productores aislados para ver sus variaciones de precio como curva

Tambien estaria bueno ver que tan esparsa es la grilla.. un grafo bipartito? para analizar

Pareciera que hay una especie de punto de quiebre alrededor del 12-15% de probabilidad de productores... valores por debajo elevan el precio de manera global, mientras que valores por encima hacen que el precio se estabilice. Parece un comportamiento monopólico, al haber pocos productores, los consumidores (que deben consumir si o sí, buscando el menor precio posible en su vecindario) están obligados a elegir entre pocos productores, que logran mayores ventas que en un escenario con alta competencia. De esta manera pueden subir los precios indiscriminadamente.

Seria interesante realizar muchas simulaciones con el mismo barrido, medir la dispersion de precios y contabilizar la cantidad de veces que esta dispersion es muy amplia, con el fin de corroborar que en los valores bajos de probabilidad de productores se alcanza el monopolio.

In [6]:
criterion2 = WithoutCriterion()
prices, probabilities = [], []
_probabilities = [value/100 for value in range(3, 25)]
repetitions = 30
length = 10

for probability in _probabilities:
    for _ in range(repetitions):
        experiment_parameters_set2 = ExperimentParametersSet(
            length=[length],
            neighborhood=[ExpandedMoore(3)],
            agent_types=[2],
            capital=[5000],
            producer_probability=[probability],
            profit_period=[7],
            price_ratio=[(1.2, 1.5)],
            fixed_cost=[(10, 1)],
            marginal_cost=[(10, 1)],
            quantity_to_buy=[(1, 0)],
            bankrupt_enabled=[False],
        )
        runner2 = Runner(Market, experiment_parameters_set2, criterion2, max_steps=500)
        runner2.start()
        final_prices = sum(runner2.experiments[0].series["price_lattice"][-1], [])
        
        prices = prices + final_prices
        probabilities = probabilities + ([probability] * length**2)

prices_dataframe = pd.DataFrame({"price": prices, "probability": probabilities})

Grafiquemos los resultados como histogramas

In [18]:
n_subplots = len(_probabilities)
fig = make_subplots(rows=n_subplots, cols=1, 
                    subplot_titles=[f'Probability: {prob}' for prob in _probabilities],
                    shared_xaxes=True,
                    vertical_spacing=0.02)

for i, prob in enumerate(_probabilities, 1):
    subset = prices_dataframe[prices_dataframe['probability'] == prob]
    fig.add_trace(
        go.Histogram(x=subset['price'], name=f'Prob {prob}'),
        row=i, col=1)

x_range = [prices_dataframe['price'].min(), prices_dataframe['price'].max()]

for i in range(1, n_subplots + 1):
    fig.update_xaxes(
        range=x_range, 
        row=i, 
        col=1,
        title_text="Price" if i == n_subplots else "",  # Only add "Price" label to the bottom subplot
        showticklabels=True,
        ticks="outside",
        ticklen=5,
        tickwidth=1,
        tickcolor='black',
        tickfont=dict(size=10))

fig.update_layout(
    height=250 * n_subplots,
    width=800, 
    title_text="Histogramas de precio por probabilidad de ocurrencia de los Productores",
    showlegend=False,
    margin=dict(t=100, b=50, l=50, r=50))
fig.show()

In [7]:
fig = px.box(prices_dataframe, x="probability", y="price")
fig.show()

---
## Trabajo futuro

#### Stock
* Los Productores actualmente poseen un atributo *stock* que decrementamos con las ventas pero que no estamos incrementando de ningún modo. Dado que el modelo permite que un Productor se endeude y siga vendiendo aún teniendo capital negativo (a menos que se active el *flag* de *"bancarrota"*), podría ser interesante que el Productor decida en que momentos permitir pérdidas o no, cuando invertir capital para recuperar stock, etc.

#### Topología
* Actualmente estamos usando una grilla única para ambos tipos de agentes, lo que nos llevó a utilizar el vecindario de Moore Extendido y tomar varias decisiones que no teníamos originalmente (Consumidores que quedan aislados sin poder acceder a Productor alguno, Productores que poseen distintas cantidades de Consumidores, etc). Esto se podría homogeneizar con una topología distinta, un modelo de doble grilla, una para cada tipo de agente, y que los mismos puedan interactuar con los 8 de enfrente (Moore clásico).

---
##